In [218]:
import sqlite3 as sq
import pandas as pd
import numpy as np
from pandas import DataFrame as df
# from interval import Interval as intt

In [233]:
# 分数预估
conn = sq.connect('高考数据库.sqlite3')     # 连接数据库
c = conn.cursor()

c.execute("SELECT name FROM sqlite_master WHERE type='table'")     # 读取表名
table = c.fetchall()
table
cates = table[10][0]     # 获取类名
mmajors = table[14][0]     # 获取专业表名
colleges = table[15][0]     # 获取学校名
province = table[12][0]
rankings = table[13][0]
fir = table[11][0]

# 获取表的列名、列名结构
def col(form):
    c.execute("SELECT * FROM {}".format(form))
    col_name = [tuple[0] for tuple in c.description]
    
    c.execute("PRAGMA table_info({})".format(form))
    col_structure = c.fetchall()
    
    return col_name,col_structure

# 获取表的内容（DF结构）
def f_df(form):
    sql = "SELECT * FROM {}".format(form)
    values = c.execute(sql)
    f_df = pd.DataFrame(data = values)
    return f_df

In [234]:
col(rankings)[0]

['id', 'year', 'score', 'rank', 'categoryID_id', 'provinceID_id']

In [355]:
# 河南一分一档表
rankings21 = f_df(rankings)
rankings21.columns = col(rankings)[0]
rankings21 = rankings21[rankings21['year'] == 2021]
rankings21 = rankings21[rankings21['provinceID_id'] == 1]
colleges21 = f_df(colleges)
colleges21[colleges21[1] == '南京大学']

,0,1,2,3,4,5
42,10284,南京大学,1,1,1,4


In [356]:
# 专业学校推荐
alldf = pd.read_csv('alldata1.0.csv',encoding='utf-8',index_col=0)

def rec(cate,score,prov,c_rank,major,m_rank,risk):
    df = alldf[alldf['categoryID_id'] == cate]     # 筛选文理科
    
    rec_rank = rankings21[rankings21['categoryID_id'] == cate]     # 河南2021一分一档对照表
    ranking = rec_rank[rec_rank['score'] == score]['rank'].values[0]
    print(ranking)
    
    df = df[df['c1'] == major[0]]
    df = df[df['c2'] == major[1]]
    df = df[df['provinceLoc'] == prov]     # 筛选城市
    
    # 统计综合得分：城市重要度和专业重要度范围为1-10
    df['sum_score'] = (df['cityScore']*c_rank + df['majorScore']*m_rank) / (c_rank + m_rank)
    print(df['collegeID_id'].values[0])
    ###############################################
    df['college'] = colleges21[colleges21[0] == df['collegeID_id'].values[0]][1].values[0]
    df = df.sort_values(['preRanking'],ascending=True)
#     print(df)
    # 报考排名范围：0.9ranking-1.5ranking
    if risk == 1:     # 输出所有符合条件的推荐方案
        df = df[df['preRanking'] <= ranking*1.5]    
        df = df[df['preRanking'] >= ranking*0.9]
    elif risk ==2:     # 输出报考风险相对大的推荐方案
#         df = df[df['preRanking'] < ranking*1.0]    
        df = df[df['preRanking'] >= ranking*0.9]
        df = df[0:6]
    else:     # 输出报考稳妥的推荐方案 
#         df = df[df['preRanking'] < ranking*1.5]    
        df = df[df['preRanking'] >= ranking*1.0]
        df = df[0:6]
    df = df.sort_values(['sum_score'],ascending=False)
    df = df.loc[:,('provinceLoc','cityLoc','college','majorName','preRanking')]
#     df = df[0:6]     # 输出前6个推荐学校及专业
    
    return df
   

In [357]:
rec(2,654,'江苏省',3,['工学','计算机类'],5,2)

3618
10284


,provinceLoc,cityLoc,college,majorName,preRanking
id,,,,,
1330480,江苏省,南京市,南京大学,计算机科学与技术,7724
1290250,江苏省,南京市,南京大学,计算机科学与技术,7724
1155877,江苏省,南京市,南京大学,计算机类,8089
1288774,江苏省,南京市,南京大学,软件工程,8900
1333489,江苏省,南京市,南京大学,软件工程,8900
969644,江苏省,苏州市,南京大学,软件工程,8900


In [350]:
alldf[alldf['collegeID_id'] == 10287]

,majorName,year,minScore,categoryID_id,collegeID_id,c1,c2,college_loc,provinceLoc,cityLoc,adcode,cityScore,c2code,majorScore,preRanking
id,,,,,,,,,,,,,,,
1288774,软件工程,2020,652,2,10287,工学,计算机类,江苏省南京市,江苏省,南京市,320100,76.91,['0812'],86.000000,8900
1290250,计算机科学与技术,2020,655,2,10287,工学,计算机类,江苏省南京市,江苏省,南京市,320100,76.91,['0812'],86.000000,7724
1291727,信息安全,2020,650,2,10287,工学,计算机类,江苏省南京市,江苏省,南京市,320100,76.91,['0812'],86.000000,9653
1293205,物联网工程,2020,649,2,10287,工学,计算机类,江苏省南京市,江苏省,南京市,320100,76.91,['0812'],86.000000,10081
1294684,人工智能,2020,650,2,10287,工学,电子信息类,江苏省南京市,江苏省,南京市,320100,76.91,"['0809', '0810']",78.000000,9653
1296164,微电子科学与工程,2020,644,2,10287,工学,电子信息类,江苏省南京市,江苏省,南京市,320100,76.91,['0809'],73.000000,12396
1297645,工程力学,2020,649,2,10287,工学,力学类,江苏省南京市,江苏省,南京市,320100,76.91,['0801'],90.000000,10081
1299127,能源与动力工程,2020,644,2,10287,工学,能源动力类,江苏省南京市,江苏省,南京市,320100,76.91,['0807'],83.000000,12396
1300610,车辆工程,2020,644,2,10287,工学,机械类,江苏省南京市,江苏省,南京市,320100,76.91,['0802'],90.000000,12396


## 用不上的函数

In [102]:
# 暂时用不上的函数
def mins(college,major):
    df = alldf[alldf['collegeID_id'] == college]
    df = df[df['c1'] == major[0]]
    df = df[df['c2'] == major[1]]
    minscore = df[df['minScore'] == min(df['minScore'])]
    return minscore

True

In [269]:
ttt = rankings21[rankings21['score'] == 750]
ttt.empty

True

In [273]:
rankings21.shape[0]

1185